### Archived Statements - part5

Target:

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 3,239,979,516 + 2,159,986,344 + 2,159,986,344 + 2,159,986,344 = 9,719,938,548

 
Process:

 - bulk import via stream consumer of import/statements_archive5
 - use externalScroller to bulk index at the same time

#### Run the imports

In [ ]:
!import.sh -o import -t 24 -l import/statements_archive5 -r archives -z 500 -b / -a -w 200000 -bulk -useScroller > logs/import-archives-step5.log

Started BulkIndex using ExternalScroller: c29c7970-c487-4841-babd-f9d2461902ff

Consumers status: threads: 24, failure 0, messages committed: 2159986560, elapsed: 171771.16s, throughput: 12574.79 msg/s


In [3]:
!tail logs/import-archives-step5.log

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [4]:
# check status
! cid="c29c7970-c487-4841-babd-f9d2461902ff"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

{"entity-type":"bulkStatus","commandId":"c29c7970-c487-4841-babd-f9d2461902ff","state":"COMPLETED","processed":2160018083,"error":false,"errorCount":0,"total":2159986560,"action":"index","username":"nco-admin","submitted":"2020-09-24T17:35:23.747Z","scrollStart":"2020-09-24T17:35:23.819Z","scrollEnd":"2020-09-26T17:18:14.970Z","processingStart":null,"processingEnd":null,"completed":"2020-09-26T17:18:14.974Z","processingMillis":0}

In [5]:
from dateutil import parser
start = "2020-09-24T17:35:23.747Z"
end = "2020-09-26T17:18:14.974Z"
nbdocs = 2159986560
s = parser.parse(start)
e = parser.parse(end) 
throughput = nbdocs / (e-s).total_seconds()

print('%s docs/s' % format(throughput, ',.2f'))

12,574.79 docs/s


In [6]:
!mongo "mongodb+srv://nxbench-2826-archives.vzeqm.mongodb.net/nuxeo" \
  --username $MONGOUSER --password $MONGOPWD \
  ../ToolBox/archives-mongodb-stats.js > snapshots/"mongodb-archives-"`date +%Y%m%d-%H%M%S`.bson

In [7]:
!statfile="snapshots/es-9B-archive-indexing-"`date +%Y%m%d-%H%M%S`.txt; \
 curl "$ES_SERVER" > $statfile; \
 curl "$ES_SERVER/_cat/health?v" >> $statfile ; \
 curl "$ES_SERVER/_cat/nodes?v" >> $statfile; \
 curl "$ES_SERVER/_cat/indices?v" >> $statfile; \
 cat $statfile

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   509  100   509    0     0  11065      0 --:--:-- --:--:-- --:--:-- 11065
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   336  100   336    0     0  24000      0 --:--:-- --:--:-- --:--:-- 24000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1403  100  1403    0     0  26980      0 --:--:-- --:--:-- --:--:-- 26980
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   864  100   864    0     0  28800      0 --:--:-- --:--:-- --:--:-- 28800
{
  "name" : "G4PH9nZ",
  "cluster_name" : "18867088